<a href="https://colab.research.google.com/github/AlperYildirim1/Pay-Attention-Later/blob/main/Inspect_Gates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "numpy<2.0.0"

In [ ]:
# @title 📊 Shimmer V5: Official Paper Evaluation (WMT14 Gold Standard)
# ==============================================================================
# 0. INSTALL & SETUP
# ==============================================================================
# Numpy fix for Colab
!pip install -q unbabel-comet bert_score x-transformers sacremoses sacrebleu huggingface_hub

import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
from bert_score import score as bert_score_func
from comet import download_model, load_from_checkpoint
import sacrebleu
import sys
import os
from huggingface_hub import hf_hub_download

# --- CONFIGURATION ---
REPO_ID = "Yujivus/PRISM-Molecule-100k"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 128
BATCH_SIZE = 32
BEAM_SIZE = 5

print(f"⚙️ Hardware: {DEVICE}")

# ==============================================================================
# 1. LOAD SHIMMER FROM HUGGING FACE
# ==============================================================================
print(f"📥 Downloading Architecture Code from {REPO_ID}...")
os.makedirs("shimmer_code", exist_ok=True)
hf_hub_download(repo_id=REPO_ID, filename="modeling_prism_gated.py", local_dir="shimmer_code")
sys.path.append("shimmer_code")

from modeling_prism_gated import PRISMHybrid_RoPE

print("📚 Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(REPO_ID)

print("🏗️ Constructing Shimmer V5...")
CONFIG = {
    "vocab_size": 58101,
    "d_model": 512,
    "num_heads": 8,
    "dff": 2048,
    "dropout": 0.1,
    "max_length": 128,
    "num_encoder_layers": 6,
    "num_refining_layers": 0,
    "num_decoder_layers": 6
}
model = PRISMHybrid_RoPE(**CONFIG)

print("📥 Downloading Weights...")
weights_path = hf_hub_download(repo_id=REPO_ID, filename="pytorch_model.bin")
state_dict = torch.load(weights_path, map_location=DEVICE)
model.load_state_dict(state_dict)

model.to(DEVICE)
model.eval()
print("✅ Shimmer V5 Ready.")


In [ ]:
# @title 🧭 Component-Wise Gate Analysis (Steering vs. Silencing)
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
# 1. SETUP
# ==============================================================================
target_word = "Bank"
context_text = "Geld Zinsen Kredit Bank" # Finans Bağlamı

gate_logs = {}
def hook_fn(name):
    def forward_hook(module, input, output):
        # Sigmoid uygulayarak 0-1 arasına çekiyoruz
        gates = torch.sigmoid(output)
        gate_logs[name] = gates.detach().cpu()
    return forward_hook

# Hook'ları Tazeleyelim
model.prism_encoder.apply(lambda m: m._forward_hooks.clear()) # Önce temizle
for i, layer in enumerate(model.prism_encoder.layers):
    layer.gate_proj.register_forward_hook(hook_fn(f"Layer_{i}"))

# ==============================================================================
# 2. ÇALIŞTIR
# ==============================================================================
inputs = tokenizer(context_text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    x = model.harmonic_embedding(inputs.input_ids)
    src_mask = (inputs.input_ids == tokenizer.pad_token_id)
    model.prism_encoder(x, src_mask)

# Hedef kelimeyi bul
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
try:
    idx = [i for i, t in enumerate(tokens) if target_word in t.replace('Ġ','')][0]
except:
    idx = 0 # Fallback

# ==============================================================================
# 3. AYRIŞTIRMA VE ANALİZ (REAL vs IMAG)
# ==============================================================================
layers = range(len(model.prism_encoder.layers))
steering_scores = []
silencing_scores = []

plt.figure(figsize=(15, 10))

for i in layers:
    # [1, Seq, D*2] -> [D*2]
    raw_gates = gate_logs[f"Layer_{i}"][0, idx, :]

    # --- KRİTİK NOKTA: CHUNK İLE AYIRMA ---
    # Modelindeki forward fonksiyonunda olduğu gibi ikiye bölüyoruz
    gate_r, gate_i = raw_gates.chunk(2, dim=-1)

    gate_r = gate_r.numpy()
    gate_i = gate_i.numpy()

    # Scatter Plot için her boyutu çiz
    # X ekseni: Real Gate, Y ekseni: Imag Gate
    plt.subplot(2, 3, i+1)

    # Renk yoğunluğu: Nöronun ne kadar aktif olduğu
    magnitude = (gate_r + gate_i) / 2

    sns.scatterplot(x=gate_r, y=gate_i, alpha=0.6, size=magnitude, sizes=(10, 100), hue=magnitude, palette="viridis", legend=False)

    # Referans Çizgileri
    plt.plot([0, 1], [0, 1], 'r--', alpha=0.3) # Diagonal (Saf Genlik Kontrolü)
    plt.title(f"Layer {i} Gate Dynamics")
    plt.xlabel("Real Gate (Pass-through)")
    plt.ylabel("Imag Gate (Rotation)")
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)

    # --- METRİK HESAPLAMA ---
    # 1. Silencing (Susturma): İkisi birden 0'a yakınsa
    # (1 - ortalama) ne kadar yüksekse o kadar susturuyor
    avg_activation = (gate_r.mean() + gate_i.mean()) / 2
    silencing_scores.append(1.0 - avg_activation)

    # 2. Steering (Döndürme): Real ve Imag arasındaki fark
    # Fark ne kadar büyükse, vektör o kadar dönüyor demektir.
    steering_avg = np.abs(gate_r - gate_i).mean()
    steering_scores.append(steering_avg)

plt.tight_layout()
plt.savefig("prism_gate_scatter.png")
plt.show()

# ==============================================================================
# 4. SONUÇ RAPORU
# ==============================================================================
print("\n📊 KATMAN KATMAN GATE STRATEJİSİ:")
print(f"{'Layer':<6} | {'Mod':<15} | {'Silencing (Susturma)':<20} | {'Steering (Döndürme)':<20}")
print("-" * 70)

for i in layers:
    sil = silencing_scores[i]
    steer = steering_scores[i]

    mode = "Pass"
    if sil > 0.8: mode = "Silence 🔇"
    elif steer > 0.15: mode = "Rotate 🔄" # Eşik değer, grafiğe göre ayarlanabilir
    elif sil < 0.3: mode = "Broadcast 📢"

    print(f"{i:<6} | {mode:<15} | {sil:.4f}{' '*14} | {steer:.4f}")

print("\n🧠 YORUM:")
print("Eğer noktalar Kırmızı Çizgi (Diagonal) üzerindeyse -> Sadece Sesi Kısıyor (Amplitude).")
print("Eğer noktalar Kırmızı Çizgiden UZAKSA -> Fazı Döndürüyor (Rotation).")